#### Small Embedding Model

In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import StaticEmbedding
from tokenizers import Tokenizer

In [2]:
embedding_tokenizer = Tokenizer.from_pretrained("google-bert/bert-base-uncased")
static_embedding = StaticEmbedding(embedding_tokenizer, embedding_dim=1024)
embedding_model = SentenceTransformer(modules=[static_embedding])

#### Load (quantized) Phi-4 for Metal (Mac Sillicon) hardware

In [3]:
from mlx_lm import load
model, tokenizer = load("mlx-community/phi-4-4bit")

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

#### Integration with LangChain

In [4]:
from langchain_community.llms.mlx_pipeline import MLXPipeline
from langchain_community.chat_models.mlx import ChatMLX
from langchain_core.messages import HumanMessage

In [5]:
llm = MLXPipeline(
    model=model, tokenizer=tokenizer, pipeline_kwargs={"max_tokens": 1024, "temp": 0.1}
)

chat = ChatMLX(llm=llm)

#### Test language model

In [4]:
messages = [
    HumanMessage(
        content="What happens when an unstoppable force meets an immovable object?"
    ),
]

# for chunk in chat.stream(messages):
#     print(chunk)
res = chat.invoke(messages)
print(res.content)

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.
As an AI, I can provide some perspectives on this classic paradox, but it's important to note that it is a thought experiment that doesn't have a definitive answer within the framework of our current understanding of physics.

The paradox of an "unstoppable force" meeting an "immovable object" presents a logical contradiction: if a force is truly unstoppable, it should be able to move any object, no matter how immovable it seems. Conversely, if an object is truly immovable, no force should be able to move it. This creates a scenario where the two definitions cannot coexist without violating the principles of logic or physics as we understand them.

In theoretical discussions, this paradox is often used to explore the limits of language, logic, and the nature of reality. It can also serve as a metaphor for situations where two seemingly irreconcilable forces or ideas come into conf

#### Ragas wrappers

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

In [7]:
evaluator_llm = LangchainLLMWrapper(chat)
evaluator_embeddings = LangchainEmbeddingsWrapper(embedding_model)

#### Evaluating Using an LLM-based Metric

In [8]:
from ragas import SingleTurnSample
from ragas.metrics import AspectCritic

test_data = {
    "user_input": "summarise given text\nThe company reported an 8% rise in Q3 2024, driven by strong performance in the Asian market. Sales in this region have significantly contributed to the overall growth. Analysts attribute this success to strategic marketing and product localization. The positive trend in the Asian market is expected to continue into the next quarter.",
    "response": "The company experienced an 8% increase in Q3 2024, largely due to effective marketing strategies and product adaptation, with expectations of continued growth in the coming quarter.",
}

metric = AspectCritic(
    name="summary_accuracy",
    llm=evaluator_llm,
    definition="Verify if the summary is accurate.",
)
test_data = SingleTurnSample(**test_data)
await metric.single_turn_ascore(test_data)

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.


1

#### Evaluating on a Dataset

In [9]:
from datasets import load_dataset
from ragas import EvaluationDataset, evaluate

eval_dataset = load_dataset(
    "explodinggradients/earning_report_summary", split="train"
).take(3)
eval_dataset = EvaluationDataset.from_hf_dataset(eval_dataset)
print("Features in dataset:", eval_dataset.features())
print("Total samples in dataset:", len(eval_dataset))

Features in dataset: ['user_input', 'response']
Total samples in dataset: 3


In [10]:
results = evaluate(eval_dataset, metrics=[metric])
results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.
[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.



{'summary_accuracy': 1.0000}